# Применение реккурентных нейронных сетей для задачи разметки текста

In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import logging
import pickle
import pymorphy2


from matplotlib import pyplot as plt

%matplotlib inline

# Load data

Загрузка текстов диалогов человека и оператора Tinkoff. Диалоги содержат слова, помеченные категориями (например: ВАЛЮТА, МЕСТО СНЯТИЯ, НАЗВАНИЕ БАНКА, ТИП КАРТЫ). Необходимо разработать алгоритм, позволяющий находить данные категории в неразмеченном диалоге.

In [5]:
with open('slotfilling-data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [8]:
def process_data(data):
    X = []
    y = []
    for item in data:
        entities = item['entities']
        if len(entities)>0:
            y_item = {}
            for entity in entities:
                y_item[entity['title']] = {
                    'start_pos': entity['start_pos'],
                    'end_pos': entity['end_pos'],
                    'text': entity['text']
                }
            X.append(item['chat'])
            y.append(y_item)
            
    
    return np.array(X), np.array(y)

In [9]:
X, y = process_data(data)

In [14]:
len(X)

3857

In [18]:
X[0],y[0]

('1: Здравствуйте. Считается ли преревод денег с карты Тинькофф на карту сбербанка операцией изъятия денежных средств?\n2: Перевод хотите осуществлять с кредитной карты?\n2: Нет, это разные операции.\n1: Какую сумму без процентов я могу перевести с-карты Тинькофф дебетовой на карту другого банка?\n2: 11 111 руб. в расчетном периоде без комиссии\n2: Свыше 11 111 руб. комиссия составит 1,1%, минимум 11 руб.\n1: Как сумму я могу перевести на карту Тинькофф без комиссии ?\n1: И какой месячный лимит снятия наличных средств с карты Тинькофф в банкоматах?\n2: Если перевод был на карту нашего банка, то это внутрибанковский перевод. по тарифу они без комиссии вне зависимости от суммы.\n2: По Вашему тарифу снимать денежные средства без комиссии Вы можете при условии снятия средств не менее 1 111 руб. за одну операция и не более 111 111 руб. за расчетный период. .\n2: Отмечу, что сторонний банк может устанавливать свою комиссию за использование его банкомата. Как правило, информация о комиссии по

In [19]:
possible_slots =np.unique(np.array([item for y_item in y for item in list(y_item.keys())]))
possible_slots

array(['ВАЛЮТА', 'ВРЕМЯ_ДАТА_СНЯТИЯ', 'ЗА_ГРАНИЦЕЙ', 'МЕСТО_СНЯТИЯ',
       'НАЗВАНИЕ_БАНКА', 'НОМЕР_ТЕЛЕФОНА', 'РАЗМЕР_КОМИССИИ',
       'СУММА_СНЯТИЯ', 'ТАРИФ_КАРТЫ', 'ТИП_КАРТЫ'], dtype='<U17')

# W2V n dataprep

Загрузка уже обученной в банке модели Word2Vec и самостоятельное обучение модели Word2Vec на корпусе из предложений, из которых состоят диалоги, для преобразования каждого слова в численный вектор. Близкие по смыслу слова преобразуются в близкие по косинусной мере вектора.

In [20]:
from gensim.models import word2vec
w2v_model = word2vec.Word2Vec.load('word2vec/w2v_model_tfidf_size300_window5_mc2.w2v')

In [21]:
import re

patt=re.compile("[a-zA-Zа-яА-Я0-9]+")

In [22]:
morph = pymorphy2.MorphAnalyzer()

Для приведения разных форм (падежей, склонений, спряжений) слов к одному виду используется лемматизация.

In [23]:
W_sent_parse=[]
for i in range(len(X)):
    S=re.split("\n1:|\n2:",X[i])
    W_sent_parse.append([morph.parse(word.lower())[0].normal_form for word in  re.findall(patt, S[0][3:])])
    for sent in range(1,len(S)):
        W_sent_parse.append([morph.parse(word.lower())[0].normal_form for word in  re.findall(patt, S[sent])])
W_sent_parse=np.array(W_sent_parse)

На получившихся корпусах предложений из нормализированных слов обучается Word2Vec.

In [24]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model_wv_parse= word2vec.Word2Vec(W_sent_parse, min_count=0,size=100, window=5, workers=2)

2018-02-13 21:15:53,976 : INFO : collecting all words and their counts
2018-02-13 21:15:53,978 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-02-13 21:15:54,010 : INFO : PROGRESS: at sentence #10000, processed 124723 words, keeping 4549 word types
2018-02-13 21:15:54,042 : INFO : PROGRESS: at sentence #20000, processed 246680 words, keeping 6347 word types
2018-02-13 21:15:54,114 : INFO : PROGRESS: at sentence #30000, processed 391620 words, keeping 7359 word types
2018-02-13 21:15:54,122 : INFO : collected 7522 word types from a corpus of 410945 raw words and 31734 sentences
2018-02-13 21:15:54,125 : INFO : Loading a fresh vocabulary
2018-02-13 21:15:54,152 : INFO : min_count=0 retains 7522 unique words (100% of original 7522, drops 0)
2018-02-13 21:15:54,153 : INFO : min_count=0 leaves 410945 word corpus (100% of original 410945, drops 0)
2018-02-13 21:15:54,196 : INFO : deleting the raw counts dictionary of 7522 items
2018-02-13 21:15:54,200 : INFO :

In [25]:
model_wv_parse.wv.most_similar_cosmul(positive=['втб'])

2018-02-13 21:16:00,347 : INFO : precomputing L2-norms of word weight vectors


[('втб11', 0.9688775539398193),
 ('сбербанковский', 0.9488134980201721),
 ('мкб', 0.948691725730896),
 ('росбанк', 0.9410445094108582),
 ('хач', 0.9386786222457886),
 ('райффайзть', 0.935751736164093),
 ('земляной', 0.9315928816795349),
 ('райффайзенбанк', 0.9306322932243347),
 ('калининград', 0.9305438995361328),
 ('пров', 0.9304080605506897)]

Создание DataFrame с номером диалога, словом, нормализованным словом, номерами начальных и конечных символов слов, обученным самостоятельно word2vec и обученным в tinkoff word2vec

In [26]:
text=[]
for t in tqdm(range(len(X))):
    for i in re.finditer(patt, X[t]):
        if i.start()>2 and ((i.start(),i.end()) not in [(j.start()+1,j.end()-1) for j in re.finditer("\n1:|\n2:", X[t])]):
            try:
                text.append((t,i.group(),morph.parse(i.group().lower())[0].normal_form,i.start(), i.end(),
                         model_wv_parse.wv[morph.parse(i.group().lower())[0].normal_form],
                         w2v_model.wv[morph.parse(i.group().lower())[0].normal_form]))
            except:
                text.append((t,i.group(),morph.parse(i.group().lower())[0].normal_form,i.start(), i.end(),
                         model_wv_parse.wv[morph.parse(i.group().lower())[0].normal_form],
                         'not_found'))

100%|██████████| 3857/3857 [06:44<00:00,  9.54it/s]


In [27]:
data_parse=pd.DataFrame(text, columns=['text','word','word_parse','start','end','my_wv','tinkoff_wv'])

In [28]:
np.array([len(sent) for sent in W_sent_parse]).sum(),len(data_parse)

(410945, 410945)

In [29]:
data_parse.head()

,text,word,word_parse,start,end,my_wv,tinkoff_wv
0,0,Здравствуйте,здравствовать,3,15,"[0.6812069, 1.5810897, 0.7941233, 0.11929614, ...","[1.4132696, 2.0395722, -0.1583433, 1.1966592, ..."
1,0,Считается,считаться,17,26,"[-0.026889628, 0.010631667, 0.3676494, 0.01146...","[1.9538485, -0.29608145, 0.44123772, 1.9070864..."
2,0,ли,ли,27,29,"[1.0615267, 0.9635242, 0.9209784, -0.5370277, ...","[0.13136159, -0.23199584, 1.2246481, -1.821377..."
3,0,преревод,преревод,30,38,"[-0.0025141998, 0.015409501, 0.0006466113, 0.0...","[-0.040791918, 0.085409895, -0.05206685, -0.11..."
4,0,денег,деньга,39,44,"[1.5603204, 0.4778464, 0.7217178, -0.5819099, ...","[-0.36978602, 2.2669377, 0.54615086, -3.661288..."


Добавление в получившуюся таблицу таргетов по всем категориям из all possible_slots

In [30]:
targets=[]
for j in range(len(possible_slots)):
    cat=possible_slots[j]
    print(cat)
    for i in range(len(y)):
        try:
            targets.append((j+1,cat,i,y[i][cat]['start_pos'],y[i][cat]['end_pos'],y[i][cat]['text']))
        except:
            continue

ВАЛЮТА
ВРЕМЯ_ДАТА_СНЯТИЯ
ЗА_ГРАНИЦЕЙ
МЕСТО_СНЯТИЯ
НАЗВАНИЕ_БАНКА
НОМЕР_ТЕЛЕФОНА
РАЗМЕР_КОМИССИИ
СУММА_СНЯТИЯ
ТАРИФ_КАРТЫ
ТИП_КАРТЫ


In [31]:
data_targ=pd.DataFrame(targets, columns=['cat_number','cat','text','start','end','word'])

In [32]:
data_parse['targ']=0

In [33]:
for i in tqdm(range(len(data_targ))):

    text=np.array(data_parse['text']==data_targ.iloc[i]['text'])
    pos=np.all([text,np.array(data_parse['start']<data_targ.iloc[i]['end']), 
                                                np.array(data_parse['end']>data_targ.iloc[i]['start'])],axis=0)
    data_parse['targ'][pos]=data_targ.iloc[i]['cat_number']
    

  0%|          | 0/8303 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 8303/8303 [10:47<00:00, 12.65it/s]


In [34]:
data_parse.head()

,text,word,word_parse,start,end,my_wv,tinkoff_wv,targ
0,0,Здравствуйте,здравствовать,3,15,"[0.6812069, 1.5810897, 0.7941233, 0.11929614, ...","[1.4132696, 2.0395722, -0.1583433, 1.1966592, ...",0
1,0,Считается,считаться,17,26,"[-0.026889628, 0.010631667, 0.3676494, 0.01146...","[1.9538485, -0.29608145, 0.44123772, 1.9070864...",0
2,0,ли,ли,27,29,"[1.0615267, 0.9635242, 0.9209784, -0.5370277, ...","[0.13136159, -0.23199584, 1.2246481, -1.821377...",0
3,0,преревод,преревод,30,38,"[-0.0025141998, 0.015409501, 0.0006466113, 0.0...","[-0.040791918, 0.085409895, -0.05206685, -0.11...",0
4,0,денег,деньга,39,44,"[1.5603204, 0.4778464, 0.7217178, -0.5819099, ...","[-0.36978602, 2.2669377, 0.54615086, -3.661288...",0


# Tokenizer

Каждому слову сопоставляется токен: 1-самое часто употребляемое слово,..., MAX_NB_WORDS(количество слов в словаре) - самое редкое слово. Предложения и диалоги превращаются в токенизированные последовательности.

In [35]:
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [36]:
MAX_NB_WORDS = len(model_wv_parse.wv.vocab)
EMBEDDING_DIM = len(model_wv_parse.wv.word_vec('1'))

In [37]:
sent_to_token=[]
for i in range(len(W_sent_parse)):  
    sent_to_token.append(" ".join(W_sent_parse[i]))

In [38]:
dialog_to_token=[]
for i in tqdm(range(data_parse['text'].max()+1)):
    dialog_to_token.append(" ".join(np.array(data_parse[data_parse['text']==i]['word_parse'])))

100%|██████████| 3857/3857 [00:07<00:00, 526.89it/s]


In [39]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS+1)
tokenizer.fit_on_texts(sent_to_token)

In [40]:
sentences = tokenizer.texts_to_sequences(sent_to_token)

In [41]:
dialogs = tokenizer.texts_to_sequences(dialog_to_token)

In [42]:
word_index = tokenizer.word_index

In [43]:
data_parse['tokenizer']=np.array([word_index[w] for w in np.array(data_parse['word_parse'])])

Для embedding слоя нейронной сети создается embedding matrix. Строка этой матрицы под номер i соответствует i-му токену-слову и представляется собой вектор, полученный при помощи word2vec, этого слова.

In [44]:
my_embedding_matrix = np.zeros((MAX_NB_WORDS+1, EMBEDDING_DIM))

In [45]:
for word, i in word_index.items():
    if word in model_wv_parse.wv.vocab:
        my_embedding_matrix[i] = model_wv_parse.wv.word_vec(word)
    else:
        print(i,word)

In [46]:
tinkoff_embedding_matrix = np.zeros((MAX_NB_WORDS, len(w2v_model.wv.word_vec('1'))))

In [47]:
rarest_word=data_parse.sort_values(by=['tokenizer'],ascending=False)[data_parse['tinkoff_wv']!='not_found']['tinkoff_wv'][0]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/ops.py:763: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = lib.scalar_compare(x, y, op)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [48]:
for word, i in word_index.items():
    if word in w2v_model.wv.vocab:
        tinkoff_embedding_matrix[i-1] = w2v_model.wv.word_vec(word)
    elif '1' in np.unique(list(word)):
        tinkoff_embedding_matrix[i-1] = w2v_model.wv.word_vec('11')
    else:
        tinkoff_embedding_matrix[i-1] = rarest_word

Таргеты предложений и диалогов также преобразуются в последовательности номеров.

In [49]:
sent_targ=[]
k=0
for i in tqdm(range(len(W_sent_parse))):
    
    sent_targ.append(np.array(data_parse[k:(k+len(W_sent_parse[i]))]['targ']))
    k=k+len(W_sent_parse[i])

100%|██████████| 31734/31734 [00:09<00:00, 3404.29it/s]


In [50]:
dialog_targ=[]
for i in tqdm(range(data_parse['text'].max())):
    dialog_targ.append(np.array(data_parse[data_parse['text']==i]['targ']))

100%|██████████| 3856/3856 [00:06<00:00, 621.48it/s]


Таблица со всеми словами и таблица с тагргетами

In [51]:
import pickle
with open('data_parse.pickle', 'wb') as f:
    pickle.dump(data_parse, f)
with open('data_targ.pickle', 'wb') as f:
    pickle.dump(data_targ, f)

Список из предложений

In [52]:
with open('W_sent_parse.pickle', 'wb') as f:
    pickle.dump(W_sent_parse, f)

Обученный word2vec

In [53]:
with open('model_wv_parse.pickle', 'wb') as f:
    pickle.dump(model_wv_parse, f)

Токенизированные предложения и диалоги и их таргеты (использовать для обучения)

In [54]:
with open('sentences.pickle', 'wb') as f:
    pickle.dump(sentences, f)
with open('dialogs.pickle', 'wb') as f:
    pickle.dump(sentences, f)
with open('sent_targ.pickle', 'wb') as f:
    pickle.dump(sent_targ, f)
with open('dialog_targ.pickle', 'wb') as f:
    pickle.dump(dialog_targ, f)

Индексы токенов и embedding matrix 

In [56]:
with open('word_index.pickle', 'wb') as f:
    pickle.dump(word_index, f)
with open('my_embedding_matrix.pickle', 'wb') as f:
    pickle.dump(my_embedding_matrix, f)
with open('tinkoff_embedding_matrix.pickle', 'wb') as f:
    pickle.dump(tinkoff_embedding_matrix, f)

# Model

Обучение модели на размеченных предложениях из первых 3000 диалогов и валидация на оставшихся диалогах.

In [92]:
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten, Dense, Input, LSTM, Embedding, Dropout, Activation, Merge, Bidirectional, SimpleRNN, GRU
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.wrappers import TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical
from catboost import CatBoostClassifier

In [93]:
with open('data_parse.pickle', 'rb') as f:
    data_parse = pickle.load(f)
with open('data_targ.pickle', 'rb') as f:
    data_targ = pickle.load(f)
with open('W_sent_parse.pickle', 'rb') as f:
    W_sent_parse = pickle.load(f)
with open('sentences.pickle', 'rb') as f:
    sentences = pickle.load(f)
with open('sent_targ.pickle', 'rb') as f:
    sent_targ = pickle.load(f)
with open('word_index.pickle', 'rb') as f:
    word_index = pickle.load(f)
with open('my_embedding_matrix.pickle', 'rb') as f:
    embedding_matrix = pickle.load(f)
with open('model_wv_parse.pickle', 'rb') as f:
    model_wv_parse = pickle.load(f)

In [94]:
MAX_NB_WORDS = embedding_matrix.shape[0]
EMBEDDING_DIM = embedding_matrix.shape[1]

In [95]:
labels=[]
for i in tqdm(range(len(sent_targ))):
    if (np.unique(sent_targ[i])).max()>1:
        labels.append(i)

100%|██████████| 31734/31734 [00:00<00:00, 46791.92it/s]


In [101]:
k=27500
s=np.array([len(n) for n in sentences[:k]]).sum()
while s<len(data_parse[data_parse['text']<3000]):
    s=s+len(sentences[k])
    k=k+1

In [105]:
train=np.array(sentences)[np.array(labels)[np.array(labels)<k]]
y_train=np.array(sent_targ)[np.array(labels)[np.array(labels)<k]]
val=np.array(sentences)[np.array(labels)[np.array(labels)>=k]]
y_val=np.array(sent_targ)[np.array(labels)[np.array(labels)>=k]]

In [106]:
y_train_cat=np.array([to_categorical(sent_label, num_classes=11).astype(int) for sent_label in list(y_train)])

В качестве модели используются нейронная сеть из нескольких слоев. Входной слой это уже обученный embedding matrix, сопоставляющий каждому токену-слову его вектор word2vec. Второй слой Bidirectional LSTM слой - реккурентный слой, обладающий длинной и короткой памятью, который позволяет обрабатывать последовательность слов в двух направлениях (прямом и обратном).

In [107]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        trainable=False))
model.add(Dropout(0.15))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.15))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.15))
model.add(TimeDistributed(Dense(11, activation='softmax')))

model.compile(loss='categorical_crossentropy',                                   
              optimizer='rmsprop',                                               
              metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/layers/recurrent.py:1993: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


Обучаем сеть на батчах, соотвествующих преддложениям из первых 3000 диалогов.

In [108]:
nb_epoch = 10
batch_size = 1
for e in range(nb_epoch):
    print("Epoch:%d" % (e+1))
    for d in tqdm(range(len(train))):
        batch_samples = np.expand_dims(train[d], axis=0)
        batch_labels =np.expand_dims(y_train_cat[d], axis=0)
        model.train_on_batch(batch_samples, batch_labels)

  0%|          | 0/3934 [00:00<?, ?it/s]

Epoch:1


  0%|          | 3/3934 [00:00<02:28, 26.42it/s]

Epoch:2


  0%|          | 3/3934 [00:00<02:29, 26.33it/s]

...

  0%|          | 2/3934 [00:00<04:11, 15.66it/s]

Epoch:9


  0%|          | 3/3934 [00:00<02:42, 24.16it/s]

Epoch:10


100%|██████████| 3934/3934 [04:18<00:00, 15.23it/s]


In [109]:
model.save('my_model.h5') 

f1-score на валидационной выборке

In [118]:
y_pred=[]
for v in tqdm(val):
    preds = model.predict(np.expand_dims(v, axis=0))
    y_pred.append(np.array([np.argmax(pred,axis=1) for pred in preds]).flatten())

100%|██████████| 967/967 [00:04<00:00, 208.13it/s]


In [126]:
from sklearn.metrics import f1_score
f1_score(np.concatenate(y_val,axis=0),np.concatenate(y_pred,axis=0), average='macro')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.5635540369077506